In [1]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import requests

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/101.0.4951.41/chromedriver_mac64.zip
Driver has been saved in cache [/Users/hollyesmith/.wdm/drivers/chromedriver/mac64/101.0.4951.41]


In [3]:
# Visit the mars nasa news site
url = 'https://redplanetscience.com'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [4]:
# set up the HTML parser

html = browser.html
news_soup = bs(html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text')

In [5]:
slide_elem.find('div', class_='content_title')

<div class="content_title">How NASA's Mars Helicopter Will Reach the Red Planet's Surface</div>

In [6]:
# Use the parent element to find the first `a` tag and save it as `news_title`
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

"How NASA's Mars Helicopter Will Reach the Red Planet's Surface"

In [7]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

'The small craft will seek to prove that powered, controlled flight is possible on another planet. But just getting it onto the surface of Mars will take a whole lot of ingenuity.'

### Featured Images

In [8]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [9]:
# Find and click the full image button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()


In [10]:
# Parse the resulting html with soup
html = browser.html
img_soup = bs(html, 'html.parser')

In [11]:
# Find the relative image url. -->.get('src') pulls the link to the image
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars1.jpg'

In [12]:
# Use the base URL to create an absolute URL. We're using an f-string for this print statement because
# it's a cleaner way to create print statements; they're also evaluated at run-time. This means that it, 
# and the variable it holds, doesn't exist until the code is executed and the values are not constant. 

img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars1.jpg'

In [13]:
# Creating a new dataframe from the HTML table. [read_html()] specifically searches for and returns
# a list of tables found in the HTML. By specifying an index of 0, we're telling Pandas to pull only 
# the first table it encounters, or the first item in the list. Then, it turns the table into a DataFrame.

df = pd.read_html('https://galaxyfacts-mars.com')[0]
df.columns=['description', 'Mars', 'Earth'] # assign columns to the new DataFrame for additional clarity

# .set_index() function turns the Description column into the DataFrame's index.
# inplace=True means that the updated index will remain in place, without having to reassign the DataFrame 
# to a new variable.
df.set_index('description', inplace=True)
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [14]:
# Pandas converts DataFrame back into HTML-ready code using [.to_html()]
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

In [15]:
# D1: Scrape High-Resolution Mars’ Hemisphere Images and Titles
# Hemispheres

# Use BeautifulSoup and Splinter to scrape full-resolution images of Mars’s hemispheres 
# and the titles of those images

In [16]:
#1. Use browser to visit the URL
url = 'https://marshemispheres.com/'
browser.visit(url)

In [17]:
#2. Create a list to hold the images and titles.
hemisphere_image_urls = []

In [18]:
#3. Write a code to retrieve the image urls and titles for each hemisphere.
html = browser.html
hemi_list = bs(html, 'html.parser')

items = hemi_list.find_all('div', class_='item')

base_url = 'https://marshemispheres.com/'

for item in items:
    url = item.find("a")['href']
    browser.visit(base_url + url)
    
    hemi_item_html = browser.html
    hemi_soup = bs(hemi_item_html, 'html.parser')
    
    title = hemi_soup.find('h2', class_ = 'title').text
    
    downloads = hemi_soup.find('div', class_ = 'downloads')
    image_url = downloads.find('a')['href']
    
    hemisphere_image_urls.append({"title": title, "img url": image_url})

In [19]:
#4. Print the list that holds the dictionary of each iage url and title.
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced', 'img url': 'images/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img url': 'images/schiaparelli_enhanced-full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img url': 'images/syrtis_major_enhanced-full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img url': 'images/valles_marineris_enhanced-full.jpg'}]

In [20]:
# 5. Quit the browser
browser.quit()